In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60301813.dfs.core.windows.net",
    "sI48e0EntAgjv6jZU0Rhqmk2WWR0vsfHYc/urFlpLKbOusdGyHZgpJjOG8ek5VCCktAYpYjh49PC+AStvm6tbg=="
)
dbutils.fs.ls("abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/")




[FileInfo(path='abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/curated/', name='curated/', size=0, modificationTime=1762882759000),
 FileInfo(path='abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/', name='gold/', size=0, modificationTime=1762596960000),
 FileInfo(path='abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/processed/', name='processed/', size=0, modificationTime=1762372579000),
 FileInfo(path='abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/raw/', name='raw/', size=0, modificationTime=1762085654000),
 FileInfo(path='abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/silver/', name='silver/', size=0, modificationTime=1762883161000)]

In [0]:
from pyspark.sql.functions import rand

# 1️⃣ Load curated reviews dataset
curated_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/curated_reviews/"
)

# 2️⃣ Shuffle the data
curated_df = curated_df.orderBy(rand())

# 3️⃣ Split into train (70%), validation (15%), test (15%)
train_df, val_df, test_df = curated_df.randomSplit([0.7, 0.15, 0.15], seed=42)

# 4️⃣ Define output paths (Gold layer / feature_v2)
base_path = "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/"
train_path = base_path + "train/"
val_path   = base_path + "val/"
test_path  = base_path + "test/"

# 5️⃣ Save splits as Delta tables
train_df.write.format("delta").mode("overwrite").save(train_path)
val_df.write.format("delta").mode("overwrite").save(val_path)
test_df.write.format("delta").mode("overwrite").save(test_path)

print("✅ Successfully split and saved Train/Validation/Test datasets to Gold layer!")

# 6️⃣ Verify counts
print(f"Train count: {train_df.count()}, Validation count: {val_df.count()}, Test count: {test_df.count()}")


✅ Successfully split and saved Train/Validation/Test datasets to Gold layer!
Train count: 5140, Validation count: 1052, Test count: 1083
